# Prepare the EDA Environment

In [1]:
import sys
import os
import types
import warnings
import glob
import regex as re
import netCDF4 as nc
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf
import ipywidgets as widgets
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import chardet
import pyperclip
from geopandas import GeoDataFrame
from shapely.geometry import Point
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
from rasterio.transform import from_origin
from rasterstats import zonal_stats
from shapely.geometry import LineString
from matplotlib.path import Path
from matplotlib.colors import Normalize
from netCDF4 import Dataset
from pyproj import CRS
from IPython.display import display
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from _windbreaks_helpers import *

In [2]:
warnings.filterwarnings('ignore')
%store -r src_dir
%store -r extent_coords
%store -r ksne_data
# %store -r stormbuf_gdf
%store -r ksne_gdf
# %store -r crop_cvr_gdf
%store -r ksne_se_count_gdf

## Set data source variables

In [3]:
# Path to the directory
directory = 'Data'

# Check if the directory exists
if os.path.isdir(directory):
    src_dir = directory
else:
    src_dir = None

print(src_dir)
%store src_dir

Data
Stored 'src_dir' (str)


## Set the extents of the Area of Interest (AOI)

In [4]:
# Project extents
extent_coords = {'min_lat': 36.9314320, 'max_lat': 43.1345169,
                 'min_lon': -104.2232889, 'max_lon': -94.5426297}
%store extent_coords

Stored 'extent_coords' (dict)


In [5]:
# Use Jupyter magik to list all variables and function loaded in the interactive workspace
%whos

Variable                     Type             Data/Info
-------------------------------------------------------
CRS                          type             <class 'pyproj.crs.crs.CRS'>
Dataset                      type             <class 'netCDF4._netCDF4.Dataset'>
GeoDataFrame                 type             <class 'geopandas.geodataframe.GeoDataFrame'>
INLINE                       Resources        Resources(mode='inline')
LATITUDE_FORMATTER           FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000002917C9ED650>
LONGITUDE_FORMATTER          FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000002917C670490>
LineString                   type             <class 'shapely.geometry.linestring.LineString'>
Normalize                    type             <class 'matplotlib.colors.Normalize'>
Path                         type             <class 'matplotlib.path.Path'>
Point                        type             <class 'shapely.geometry.point.Point'>
ccrs                    

## Load basemaps and boundary files for AOI

# Examine Storm Event Data

## - Examine Storm Event csv's

In [6]:
# prefix = 'StormEvents_KSNE'
# combined_gdf = combine_csv_files(src_dir+'/Storm_event/', prefix)

In [7]:
# combined_gdf.head()

In [8]:
# print_cols(combined_gdf)

In [9]:
# combined_gdf.to_csv(os.path.join(src_dir, 'Storm_event/StormEvents_all.csv'))

In [10]:
ksne_data = pd.read_csv(os.path.join(src_dir, 'Storm_event/StormEvents_all_xlmod.csv'))
%store ksne_data

Stored 'ksne_data' (DataFrame)


In [11]:
ksne_data.head()

,BEGIN_YEARMONTH,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,FIPS,COUNTY_NAME,MAGNITUDE,MAGNITUDE_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,geometry
0,201106,320901,KANSAS,20,2011,June,Thunderstorm Wind,20115,MARION,52.0,EG,38.3500,-97.2000,38.3500,-97.2000,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,NaN
1,201106,320907,KANSAS,20,2011,June,Thunderstorm Wind,20015,BUTLER,52.0,EG,37.7000,-97.1500,37.7000,-97.1500,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,NaN
2,201107,317493,NEBRASKA,31,2011,July,Thunderstorm Wind,31161,SHERIDAN,52.0,EG,42.6529,-102.2829,42.6529,-102.2829,Isolated thunderstorms developed in the Nebras...,Thunderstorm wind gusts tore 4 inch branches o...,NaN
3,201106,329466,KANSAS,20,2011,June,Thunderstorm Wind,20025,CLARK,78.0,EG,37.3900,-100.0100,37.3900,-100.0100,High terrain thunderstorms drifted out of east...,Tree branches of 4 to 5 inches in diameter wer...,NaN
4,201106,329467,KANSAS,20,2011,June,Thunderstorm Wind,20069,GRAY,52.0,EG,37.9700,-100.3100,37.9700,-100.3100,High terrain thunderstorms drifted out of east...,There was extensive blowing dust with the wind.,NaN


In [12]:
ksne_data['MONTH'] = ksne_data['MONTH_NAME'].apply(month_name_to_number)
ksne_data.head()

,BEGIN_YEARMONTH,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,FIPS,COUNTY_NAME,MAGNITUDE,MAGNITUDE_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,geometry,MONTH
0,201106,320901,KANSAS,20,2011,June,Thunderstorm Wind,20115,MARION,52.0,EG,38.3500,-97.2000,38.3500,-97.2000,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,NaN,6
1,201106,320907,KANSAS,20,2011,June,Thunderstorm Wind,20015,BUTLER,52.0,EG,37.7000,-97.1500,37.7000,-97.1500,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,NaN,6
2,201107,317493,NEBRASKA,31,2011,July,Thunderstorm Wind,31161,SHERIDAN,52.0,EG,42.6529,-102.2829,42.6529,-102.2829,Isolated thunderstorms developed in the Nebras...,Thunderstorm wind gusts tore 4 inch branches o...,NaN,7
3,201106,329466,KANSAS,20,2011,June,Thunderstorm Wind,20025,CLARK,78.0,EG,37.3900,-100.0100,37.3900,-100.0100,High terrain thunderstorms drifted out of east...,Tree branches of 4 to 5 inches in diameter wer...,NaN,6
4,201106,329467,KANSAS,20,2011,June,Thunderstorm Wind,20069,GRAY,52.0,EG,37.9700,-100.3100,37.9700,-100.3100,High terrain thunderstorms drifted out of east...,There was extensive blowing dust with the wind.,NaN,6


In [13]:
print_cols(ksne_data)


 Dataframe columns: 

Index: 0, Column Name: BEGIN_YEARMONTH, Data Type: int64
Index: 1, Column Name: EVENT_ID, Data Type: int64
Index: 2, Column Name: STATE, Data Type: object
Index: 3, Column Name: STATE_FIPS, Data Type: int64
Index: 4, Column Name: YEAR, Data Type: int64
Index: 5, Column Name: MONTH_NAME, Data Type: object
Index: 6, Column Name: EVENT_TYPE, Data Type: object
Index: 7, Column Name: FIPS, Data Type: int64
Index: 8, Column Name: COUNTY_NAME, Data Type: object
Index: 9, Column Name: MAGNITUDE, Data Type: float64
Index: 10, Column Name: MAGNITUDE_TYPE, Data Type: object
Index: 11, Column Name: BEGIN_LAT, Data Type: float64
Index: 12, Column Name: BEGIN_LON, Data Type: float64
Index: 13, Column Name: END_LAT, Data Type: float64
Index: 14, Column Name: END_LON, Data Type: float64
Index: 15, Column Name: EPISODE_NARRATIVE, Data Type: object
Index: 16, Column Name: EVENT_NARRATIVE, Data Type: object
Index: 17, Column Name: geometry, Data Type: float64
Index: 18, Column Name

In [14]:
# ksne_data.drop(columns=['geometry', 'STATE_FIPS'])
# Rename long-name fields to <= 10 character for shp file constraints
ksne_data = ksne_data.rename(columns={'BEGIN_YEARMONTH': 'YRMO', 'MONTH_NAME': 'MO_NAME', 'COUNTY_NAME': 'CO_NAME', 'MAGNITUDE': 'MAG', 'MAGNITUDE_TYPE':'MAG_TYPE', 'EPISODE_NARRATIVE':'EP_NARR', 'EVENT_NARRATIVE':'EVENT_NARR'})
print_cols(ksne_data)


 Dataframe columns: 

Index: 0, Column Name: YRMO, Data Type: int64
Index: 1, Column Name: EVENT_ID, Data Type: int64
Index: 2, Column Name: STATE, Data Type: object
Index: 3, Column Name: STATE_FIPS, Data Type: int64
Index: 4, Column Name: YEAR, Data Type: int64
Index: 5, Column Name: MO_NAME, Data Type: object
Index: 6, Column Name: EVENT_TYPE, Data Type: object
Index: 7, Column Name: FIPS, Data Type: int64
Index: 8, Column Name: CO_NAME, Data Type: object
Index: 9, Column Name: MAG, Data Type: float64
Index: 10, Column Name: MAG_TYPE, Data Type: object
Index: 11, Column Name: BEGIN_LAT, Data Type: float64
Index: 12, Column Name: BEGIN_LON, Data Type: float64
Index: 13, Column Name: END_LAT, Data Type: float64
Index: 14, Column Name: END_LON, Data Type: float64
Index: 15, Column Name: EP_NARR, Data Type: object
Index: 16, Column Name: EVENT_NARR, Data Type: object
Index: 17, Column Name: geometry, Data Type: float64
Index: 18, Column Name: MONTH, Data Type: int64



In [15]:
cp1_ksne_data = ksne_data
# st_abb dictionary
state_abb = {'KANSAS': 'KS', 'NEBRASKA': 'NE'}

ksne_data['ST_ABB'] = ksne_data['STATE'].map(state_abb).astype('object')
ksne_data.head()

,YRMO,EVENT_ID,STATE,STATE_FIPS,YEAR,MO_NAME,EVENT_TYPE,FIPS,CO_NAME,MAG,MAG_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EP_NARR,EVENT_NARR,geometry,MONTH,ST_ABB
0,201106,320901,KANSAS,20,2011,June,Thunderstorm Wind,20115,MARION,52.0,EG,38.3500,-97.2000,38.3500,-97.2000,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,NaN,6,KS
1,201106,320907,KANSAS,20,2011,June,Thunderstorm Wind,20015,BUTLER,52.0,EG,37.7000,-97.1500,37.7000,-97.1500,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,NaN,6,KS
2,201107,317493,NEBRASKA,31,2011,July,Thunderstorm Wind,31161,SHERIDAN,52.0,EG,42.6529,-102.2829,42.6529,-102.2829,Isolated thunderstorms developed in the Nebras...,Thunderstorm wind gusts tore 4 inch branches o...,NaN,7,NE
3,201106,329466,KANSAS,20,2011,June,Thunderstorm Wind,20025,CLARK,78.0,EG,37.3900,-100.0100,37.3900,-100.0100,High terrain thunderstorms drifted out of east...,Tree branches of 4 to 5 inches in diameter wer...,NaN,6,KS
4,201106,329467,KANSAS,20,2011,June,Thunderstorm Wind,20069,GRAY,52.0,EG,37.9700,-100.3100,37.9700,-100.3100,High terrain thunderstorms drifted out of east...,There was extensive blowing dust with the wind.,NaN,6,KS


In [16]:
print_cols(ksne_data)


 Dataframe columns: 

Index: 0, Column Name: YRMO, Data Type: int64
Index: 1, Column Name: EVENT_ID, Data Type: int64
Index: 2, Column Name: STATE, Data Type: object
Index: 3, Column Name: STATE_FIPS, Data Type: int64
Index: 4, Column Name: YEAR, Data Type: int64
Index: 5, Column Name: MO_NAME, Data Type: object
Index: 6, Column Name: EVENT_TYPE, Data Type: object
Index: 7, Column Name: FIPS, Data Type: int64
Index: 8, Column Name: CO_NAME, Data Type: object
Index: 9, Column Name: MAG, Data Type: float64
Index: 10, Column Name: MAG_TYPE, Data Type: object
Index: 11, Column Name: BEGIN_LAT, Data Type: float64
Index: 12, Column Name: BEGIN_LON, Data Type: float64
Index: 13, Column Name: END_LAT, Data Type: float64
Index: 14, Column Name: END_LON, Data Type: float64
Index: 15, Column Name: EP_NARR, Data Type: object
Index: 16, Column Name: EVENT_NARR, Data Type: object
Index: 17, Column Name: geometry, Data Type: float64
Index: 18, Column Name: MONTH, Data Type: int64
Index: 19, Column N

In [17]:
# List of columns in the desired order
column_order = (
        ['YRMO', 'FIPS', 'YEAR', 'MONTH', 'MO_NAME', 'ST_ABB', 'CO_NAME', 'EVENT_ID', 'EVENT_TYPE'] +
        [
            c for c in ksne_data.columns
            if c not in {
            'YRMO',
            'FIPS',
            'YEAR',
            'MONTH',
            'MO_NAME',
            'ST_ABB',
            'CO_NAME',
            'EVENT_ID',
            'EVENT_TYPE'
        }
        ]
)
# Create a new DataFrame with columns in the desired order
ksne_data = ksne_data[column_order]

# Print the columns of the new DataFrame
print_cols(ksne_data)


 Dataframe columns: 

Index: 0, Column Name: YRMO, Data Type: int64
Index: 1, Column Name: FIPS, Data Type: int64
Index: 2, Column Name: YEAR, Data Type: int64
Index: 3, Column Name: MONTH, Data Type: int64
Index: 4, Column Name: MO_NAME, Data Type: object
Index: 5, Column Name: ST_ABB, Data Type: object
Index: 6, Column Name: CO_NAME, Data Type: object
Index: 7, Column Name: EVENT_ID, Data Type: int64
Index: 8, Column Name: EVENT_TYPE, Data Type: object
Index: 9, Column Name: STATE, Data Type: object
Index: 10, Column Name: STATE_FIPS, Data Type: int64
Index: 11, Column Name: MAG, Data Type: float64
Index: 12, Column Name: MAG_TYPE, Data Type: object
Index: 13, Column Name: BEGIN_LAT, Data Type: float64
Index: 14, Column Name: BEGIN_LON, Data Type: float64
Index: 15, Column Name: END_LAT, Data Type: float64
Index: 16, Column Name: END_LON, Data Type: float64
Index: 17, Column Name: EP_NARR, Data Type: object
Index: 18, Column Name: EVENT_NARR, Data Type: object
Index: 19, Column Name

In [18]:
%store ksne_data
ksne_data.to_csv(os.path.join(src_dir, 'Storm_event/StormEvents_all_pdmod.csv'))

Stored 'ksne_data' (DataFrame)


In [19]:
source = ksne_data
output = 'Output/stormevents_ksne.shp'

ksne_gdf = df_gdf_or_csv_to_shp(source, output, extent_coords)

In [20]:
# Group by 'YEARMO' and FIPS and aggregate
# ksne_gdf = ksne_gdf.groupby(['YEARMO', 'FIPS']).sum().reset_index()

min_value = ksne_gdf['YRMO'].min()
max_value = ksne_gdf['YRMO'].max()

print('Minimum value in YRMO column:', min_value)
print('Maximum value in YRMO column:', max_value)


Minimum value in YRMO column: 201102
Maximum value in YRMO column: 202011


In [21]:

# Specify the path to the input geodataframe
input_gdf = ksne_gdf
# Specify the buffer distance as a float or int
buf_dist = 3
# or you can specify a str if you want to buffer by meters or feet instead of kilometers and miles, respectively
# buf_dist = '3000ft'
buf_dist_str = str(buf_dist).replace('.', '_')
# Specify the path to the output shapefile
output_path = f'Output/storm_line_{buf_dist_str}_buf.shp'

# Create a buffered GeoDataFrame
# Note: For Geographic Coordinate Systems, provide buffer distance in kilometers.
# For State Plane Coordinate Systems, provide buffer distance in miles.
gdf_buf(input_gdf, buf_dist, output_path)


,YRMO,FIPS,YEAR,MONTH,MO_NAME,ST_ABB,CO_NAME,EVENT_ID,EVENT_TYPE,STATE,STATE_FIPS,MAG,MAG_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EP_NARR,EVENT_NARR,geometry
0,201106,20115,2011,6,June,KS,MARION,320901,Thunderstorm Wind,KANSAS,20,52.0,EG,38.3500,-97.2000,38.3500,-97.2000,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,"POLYGON ((-97.17305 38.35000, -97.17318 38.347..."
1,201106,20015,2011,6,June,KS,BUTLER,320907,Thunderstorm Wind,KANSAS,20,52.0,EG,37.7000,-97.1500,37.7000,-97.1500,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,"POLYGON ((-97.12305 37.70000, -97.12318 37.697..."
2,201107,31161,2011,7,July,NE,SHERIDAN,317493,Thunderstorm Wind,NEBRASKA,31,52.0,EG,42.6529,-102.2829,42.6529,-102.2829,Isolated thunderstorms developed in the Nebras...,Thunderstorm wind gusts tore 4 inch branches o...,"POLYGON ((-102.25595 42.65290, -102.25608 42.6..."
3,201106,20025,2011,6,June,KS,CLARK,329466,Thunderstorm Wind,KANSAS,20,78.0,EG,37.3900,-100.0100,37.3900,-100.0100,High terrain thunderstorms drifted out of east...,Tree branches of 4 to 5 inches in diameter wer...,"POLYGON ((-99.98305 37.39000, -99.98318 37.387..."
4,201106,20069,2011,6,June,KS,GRAY,329467,Thunderstorm Wind,KANSAS,20,52.0,EG,37.9700,-100.3100,37.9700,-100.3100,High terrain thunderstorms drifted out of east...,There was extensive blowing dust with the wind.,"POLYGON ((-100.28305 37.97000, -100.28318 37.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14404,202006,20091,2020,6,June,KS,JOHNSON,906925,Thunderstorm Wind,KANSAS,20,52.0,EG,39.0000,-94.6500,39.0000,-94.6500,Late in the evening and overnight hours a stro...,A large tree branch around 1 foot in diameter ...,"POLYGON ((-94.62305 39.00000, -94.62318 38.997..."
14405,202006,20103,2020,6,June,KS,LEAVENWORTH,906926,Thunderstorm Wind,KANSAS,20,53.0,MG,39.1100,-95.0800,39.1100,-95.0800,Late in the evening and overnight hours a stro...,Emergency Management reported 61 mph gust in T...,"POLYGON ((-95.05305 39.11000, -95.05318 39.107..."
14406,202006,20209,2020,6,June,KS,WYANDOTTE,906923,Thunderstorm Wind,KANSAS,20,52.0,EG,39.1300,-94.8300,39.1300,-94.8300,Late in the evening and overnight hours a stro...,NWS employee reported their patio table was fl...,"POLYGON ((-94.80305 39.13000, -94.80318 39.127..."
14407,202006,20103,2020,6,June,KS,LEAVENWORTH,896590,Thunderstorm Wind,KANSAS,20,56.0,EG,39.3100,-94.9400,39.3100,-94.9400,As part of a larger severe weather event some ...,Emergency management reported a tree down on a...,"POLYGON ((-94.91305 39.31000, -94.91318 39.307..."


In [22]:
# # I need crop cover
# # Load the storm line data from a shapefile
# buf_dist_str = '3'
# stormbuf_gdf = gpd.read_file(f'Output/storm_line_{buf_dist_str}_buf.shp')
# # Perform the spatial join (intersection)
# intersect_gdf = gpd.sjoin(crop_cvr_gdf, stormbuf_gdf, how='inner', op='intersects')
# intersect_gdf.head()

In [23]:
# # get column names with their index
# print('\n', 'stormbuf_gdf columns:', '\n')
# print_cols(stormbuf_gdf)

In [24]:
# print('crop_cvr_gdf', ksne_gdf.crs)
# print('ksne_gdf', ksne_gdf.crs)
# print('stormbuf_gdf', stormbuf_gdf.crs)

In [25]:
%whos

Variable                     Type             Data/Info
-------------------------------------------------------
CRS                          type             <class 'pyproj.crs.crs.CRS'>
Dataset                      type             <class 'netCDF4._netCDF4.Dataset'>
GeoDataFrame                 type             <class 'geopandas.geodataframe.GeoDataFrame'>
INLINE                       Resources        Resources(mode='inline')
LATITUDE_FORMATTER           FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000002917C9ED650>
LONGITUDE_FORMATTER          FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000002917C670490>
LineString                   type             <class 'shapely.geometry.linestring.LineString'>
Normalize                    type             <class 'matplotlib.colors.Normalize'>
Path                         type             <class 'matplotlib.path.Path'>
Point                        type             <class 'shapely.geometry.point.Point'>
buf_dist                

In [26]:
ksne_gdf.head()

,YRMO,FIPS,YEAR,MONTH,MO_NAME,ST_ABB,CO_NAME,EVENT_ID,EVENT_TYPE,STATE,STATE_FIPS,MAG,MAG_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EP_NARR,EVENT_NARR,geometry
0,201106,20115,2011,6,June,KS,MARION,320901,Thunderstorm Wind,KANSAS,20,52.0,EG,38.3500,-97.2000,38.3500,-97.2000,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,"LINESTRING (-97.20000 38.35000, -97.20000 38.3..."
1,201106,20015,2011,6,June,KS,BUTLER,320907,Thunderstorm Wind,KANSAS,20,52.0,EG,37.7000,-97.1500,37.7000,-97.1500,A Squall Line developed along the Interstate 1...,Estimated gust reported by a trained spotter.,"LINESTRING (-97.15000 37.70000, -97.15000 37.7..."
2,201107,31161,2011,7,July,NE,SHERIDAN,317493,Thunderstorm Wind,NEBRASKA,31,52.0,EG,42.6529,-102.2829,42.6529,-102.2829,Isolated thunderstorms developed in the Nebras...,Thunderstorm wind gusts tore 4 inch branches o...,"LINESTRING (-102.28290 42.65290, -102.28290 42..."
3,201106,20025,2011,6,June,KS,CLARK,329466,Thunderstorm Wind,KANSAS,20,78.0,EG,37.3900,-100.0100,37.3900,-100.0100,High terrain thunderstorms drifted out of east...,Tree branches of 4 to 5 inches in diameter wer...,"LINESTRING (-100.01000 37.39000, -100.01000 37..."
4,201106,20069,2011,6,June,KS,GRAY,329467,Thunderstorm Wind,KANSAS,20,52.0,EG,37.9700,-100.3100,37.9700,-100.3100,High terrain thunderstorms drifted out of east...,There was extensive blowing dust with the wind.,"LINESTRING (-100.31000 37.97000, -100.31000 37..."


In [27]:
print("Unique Years/Months: ", ksne_gdf['YRMO'].unique())
print("Unique Counties by FIPS: ", ksne_gdf['FIPS'].unique())


Unique Years/Months:  [201106 201107 201105 201108 201104 201110 201109 201102 201103 201112
 201211 201210 201204 201205 201202 201203 201209 201206 201207 201208
 201306 201305 201310 201307 201308 201304 201309 201301 201303 201408
 201409 201406 201410 201407 201412 201405 201404 201509 201504 201506
 201507 201505 201508 201511 201503 201510 201607 201606 201605 201608
 201612 201604 201603 201609 201610 201611 201704 201706 201705 201703
 201707 201702 201708 201709 201710 201806 201805 201807 201803 201811
 201804 201809 201810 201808 201905 201906 201904 201907 201903 201908
 201909 201910 201912 202007 202006 202008 202009 202010 202005 202004
 202011]
Unique Counties by FIPS:  [20115 20015 31161 20025 20069 20065 20083 20057 20097 20151 20145 20035
 20017 20167 20159 20113 31049 20049 20001 20125 31031 20079 31101 20193
 20093 20207 20019 20067 20033 20101 20055 31157 31041 31009 31149 31089
 31057 20075 20119 31067 31013 31003 20133 20099 20135 20051 31039 20047
 20171 31005

In [33]:
ksne_se_count = ksne_gdf.groupby(['YRMO', 'FIPS']).size().reset_index(name='EVENT_COUNT')
# Save the DataFrame to a CSV
ksne_se_count.to_csv('Output/ksne_se_count.csv', index=False)
print(ksne_se_count)

        YRMO   FIPS  EVENT_COUNT
0     201102  20003            1
1     201102  20019            1
2     201102  20035            1
3     201102  20037            2
4     201102  20125            1
...      ...    ...          ...
4599  202011  20007            2
4600  202011  20099            1
4601  202011  20125            1
4602  202011  20173            2
4603  202011  20191            1

[4604 rows x 3 columns]


In [35]:
ksne_se_count = pd.read_csv('Output/ksne_se_count.csv')

print_cols(ksne_se_count)


 Dataframe columns: 

Index: 0, Column Name: YRMO, Data Type: datetime64[ns]
Index: 1, Column Name: FIPS, Data Type: int64
Index: 2, Column Name: EVENT_COUNT, Data Type: int64



# Store notebook variables for use in other notebooks in the project.

In [36]:
%store src_dir
%store extent_coords
%store ksne_data
# %store stormbuf_gdf
%store ksne_gdf
# %store crop_cvr_gdf
%store ksne_se_count

Stored 'src_dir' (str)
Stored 'extent_coords' (dict)
Stored 'ksne_data' (DataFrame)
Stored 'ksne_gdf' (GeoDataFrame)
Stored 'ksne_se_count' (DataFrame)
